Imputation used for test data is same as training data

Median based imputation is done using actual train data

Using Logistic Regression

In [1]:
#import all the necessary libraries

import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import confusion_matrix,roc_auc_score,roc_curve,auc
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,classification_report
import seaborn as sns
from sklearn.decomposition import pca
from datetime import datetime
from collections import Counter
from sklearn.decomposition import pca
from datetime import datetime
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import pickle


## 1. Reading data :

### 1.1 Test Data

In [2]:
pd.set_option('display.max_columns',None)
test_data = pd.read_csv('aps_failure_test_set.csv',nrows=1)
test_data

,class,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,ag_003,ag_004,ag_005,ag_006,ag_007,ag_008,ag_009,ah_000,ai_000,aj_000,ak_000,al_000,am_0,an_000,ao_000,ap_000,aq_000,ar_000,as_000,at_000,au_000,av_000,ax_000,ay_000,ay_001,ay_002,ay_003,ay_004,ay_005,ay_006,ay_007,ay_008,ay_009,az_000,az_001,az_002,az_003,az_004,az_005,az_006,az_007,az_008,az_009,ba_000,ba_001,ba_002,ba_003,ba_004,ba_005,ba_006,ba_007,ba_008,ba_009,bb_000,bc_000,bd_000,be_000,bf_000,bg_000,bh_000,bi_000,bj_000,bk_000,bl_000,bm_000,bn_000,bo_000,bp_000,bq_000,br_000,bs_000,bt_000,bu_000,bv_000,bx_000,by_000,bz_000,ca_000,cb_000,cc_000,cd_000,ce_000,cf_000,cg_000,ch_000,ci_000,cj_000,ck_000,cl_000,cm_000,cn_000,cn_001,cn_002,cn_003,cn_004,cn_005,cn_006,cn_007,cn_008,cn_009,co_000,cp_000,cq_000,cr_000,cs_000,cs_001,cs_002,cs_003,cs_004,cs_005,cs_006,cs_007,cs_008,cs_009,ct_000,cu_000,cv_000,cx_000,cy_000,cz_000,da_000,db_000,dc_000,dd_000,de_000,df_000,dg_000,dh_000,di_000,dj_000,dk_000,dl_000,dm_000,dn_000,do_000,dp_000,dq_000,dr_000,ds_000,dt_000,du_000,dv_000,dx_000,dy_000,dz_000,ea_000,eb_000,ec_00,ed_000,ee_000,ee_001,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
0,neg,60,0,20,12,0,0,0,0,0,2682,4736,3862,1846,0,0,0,3976,0,0,0,1520,2374,11516,9480,111258,470,0,0,0,0,58,26,0,0,0,0,0,0,0,0,13124,2,1956,434,76,174,190,6198,1148,2948,2,0,8762,2566,480,380,196,516,86,66,74,0,124340,4,6,4,0,3976,318,107662,3458,na,na,na,na,na,na,na,na,110980,59.53,124340,124340,184970,54,686,12246,123880,13196,1209600,598,0,6,0,5913.6,0,5851.2,6,30,0,2,5512,4674,1388,1508,38,4,0,0,8,12,124340,0,1550,14,36,26,920,430,7650,2294,206,0,22,42,5336,1276,0,0,0,0,6598,70,112,0,0,0,0,0,0,0,0,340,0,0,1100,574,232,66,780,882,0,4,0,0,0,465.5,90,7502,3156,1098,138,412,654,78,88,0,0,0,0


### 1.2 Processing Class data

In [3]:
#test_data.replace(to_replace= ['nan','na','NA','n.a.','?','--','n/a'],value=np.NaN,inplace=True)

test_data['class'] = test_data['class'].map({'neg':0,'pos':1})

y_test = test_data['class']
test_data.drop(['class'],axis=1,inplace=True)

test_data

,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,ag_003,ag_004,ag_005,ag_006,ag_007,ag_008,ag_009,ah_000,ai_000,aj_000,ak_000,al_000,am_0,an_000,ao_000,ap_000,aq_000,ar_000,as_000,at_000,au_000,av_000,ax_000,ay_000,ay_001,ay_002,ay_003,ay_004,ay_005,ay_006,ay_007,ay_008,ay_009,az_000,az_001,az_002,az_003,az_004,az_005,az_006,az_007,az_008,az_009,ba_000,ba_001,ba_002,ba_003,ba_004,ba_005,ba_006,ba_007,ba_008,ba_009,bb_000,bc_000,bd_000,be_000,bf_000,bg_000,bh_000,bi_000,bj_000,bk_000,bl_000,bm_000,bn_000,bo_000,bp_000,bq_000,br_000,bs_000,bt_000,bu_000,bv_000,bx_000,by_000,bz_000,ca_000,cb_000,cc_000,cd_000,ce_000,cf_000,cg_000,ch_000,ci_000,cj_000,ck_000,cl_000,cm_000,cn_000,cn_001,cn_002,cn_003,cn_004,cn_005,cn_006,cn_007,cn_008,cn_009,co_000,cp_000,cq_000,cr_000,cs_000,cs_001,cs_002,cs_003,cs_004,cs_005,cs_006,cs_007,cs_008,cs_009,ct_000,cu_000,cv_000,cx_000,cy_000,cz_000,da_000,db_000,dc_000,dd_000,de_000,df_000,dg_000,dh_000,di_000,dj_000,dk_000,dl_000,dm_000,dn_000,do_000,dp_000,dq_000,dr_000,ds_000,dt_000,du_000,dv_000,dx_000,dy_000,dz_000,ea_000,eb_000,ec_00,ed_000,ee_000,ee_001,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
0,60,0,20,12,0,0,0,0,0,2682,4736,3862,1846,0,0,0,3976,0,0,0,1520,2374,11516,9480,111258,470,0,0,0,0,58,26,0,0,0,0,0,0,0,0,13124,2,1956,434,76,174,190,6198,1148,2948,2,0,8762,2566,480,380,196,516,86,66,74,0,124340,4,6,4,0,3976,318,107662,3458,na,na,na,na,na,na,na,na,110980,59.53,124340,124340,184970,54,686,12246,123880,13196,1209600,598,0,6,0,5913.6,0,5851.2,6,30,0,2,5512,4674,1388,1508,38,4,0,0,8,12,124340,0,1550,14,36,26,920,430,7650,2294,206,0,22,42,5336,1276,0,0,0,0,6598,70,112,0,0,0,0,0,0,0,0,340,0,0,1100,574,232,66,780,882,0,4,0,0,0,465.5,90,7502,3156,1098,138,412,654,78,88,0,0,0,0


## 2. Data Preprocessing

### 2.1 Missing data preprocessing

In [4]:
def missing_value(data):
    
    #Calculating the number of missing values
    missing_value = data.isnull().sum()
    # sorting features with missing value : highest to lowest
    missing_col = missing_value.sort_values(ascending = False)
        #print('Sorted data based on missing values : \n',missing_col)
    
    #Processing missing values
    
    # Code to identify rows which contains features(less than 5% missing values)

    missing_rvalue = data.isnull().sum(axis=1)
    missing_row = missing_rvalue.sort_values(ascending = False)
    #print(missing_row)
    
    missing_per = (missing_col/len(data))*100
    print('Missing value percentage of train data: \n',missing_per)
    
    return missing_per
    

### 2.2 Median based imputation

In [5]:
#Total_cost = get_confusion_matrix(y_test,y_pred_test)
def impute_model(test_data1):
    
    
    train_data = pd.read_csv('processed_train_data_final.csv')
    
    test_data2 = test_data1.fillna(train_data.median())
    
    return test_data2

__Observations : 1. We imputed all the features with missing values with median values__

__2. Features with missing values are imputed using train dataset__

__3. Using train dataset help us impute even one single row effectively__

## 3 Modelling data

### 3.1 Resampling data

In [6]:
def get_resampling(df_test):
    
    
    filename = 'resamplingLR80.sav'
    sample_model = pickle.load(open(filename,'rb'))
    estimators= len(sample_model)
    pred_test_final = []
    
    for iters in range(estimators):
    
        # Calculating and storing variable of test dataset individually in a list
        model = sample_model[iters]
        y_test_pred = model.predict(df_test)
        pred_test_final.append(y_test_pred)
        
    
    # Transposing the predicted dataset
    pred_test_final = np.array(pred_test_final).T
    #print(pred_test_final)
    
    return pred_test_final
    

### 3.2 Plotting confusion matrix

In [7]:
def get_confusion_matrix(y_test,y_pred_test):
    
    #ROC AUC score
    #auc = roc_auc_score(y_test,y_pred_test)
    #print('AUC score: %.3f' % auc)
 
    accuracy = accuracy_score(y_test,y_pred_test)
    print('Accuracy : %.3f' % accuracy)
    #precision
    precision = precision_score(y_test,y_pred_test)
    print('Precision : %.3f' % precision)
    #Recall
    recal = recall_score(y_test,y_pred_test)
    print('Recall : %.3f' % recal)
    #f1 score
    f1 = f1_score(y_test,y_pred_test)
    print('f1_score : %.3f' % f1)
    
    #confusion matrix
    cm_test = confusion_matrix(y_test,y_pred_test)
    #print('Confusion Matrix :\n', matrix)
    print('-'*120)
    print('Dimension of Confusion Matrix : ',len(cm_test))
    print('-'*120)
    print('Confusion Matrix :','\n',cm_test)
    print('-'*120)
    if(len(cm_test) == 2):
        
        print("Type 1 error (False Positives) : ",cm_test[0][1])
        print('Type 2 error (False Negatives) :',cm_test[1][0])
        print('-'*120)
        Total_cost = cm_test[0][1]*10 + cm_test[1][0]*500
        print('Total cost : ',Total_cost)
        print('-'*120)
    
        return Total_cost
    
    else :
        print("Test data contains only 1 type of data")
        Total_cost = 0
        print('Total cost : ',Total_cost)
        return Total_cost

### 3.3 Machine Learning Model : Function 1

In [8]:
def final_func_1(test_data1):
    
    #Data Preprocessing
    
    # processing all missing data and combining them in one format
    test_data1.replace(to_replace = ['nan','na','NA','n.a.','?','--','n/a'], value = np.NaN, inplace = True)
    
    #imputing missing values using median imputation
    test_data1 = impute_model(test_data1)
    
    #imputing test values
    testdf = get_resampling(test_data1)
    
    #Converting result(meta model) into dataframe
    test_meta = pd.DataFrame(testdf)
    
    
    #Applying the pretrained model
    
    filename = 'Model_LR80.sav'

    loaded_model = pickle.load(open(filename,'rb'))

    y_pred_test = loaded_model.predict(test_meta)
    
    return y_pred_test

In [9]:
# predicting the test data
y_pred =  final_func_1(test_data)

__Observation : 1. In function, we predicted the output values inputing just test data__

__2. The functions works well irrespective of size of data__

__3. We have used 2 pretrained model, one  is for resampling and second one is for machine learning model__

__4. Logistic Regression with balanced class weight is imported for evaluating the test dataset__

__5. The performance of data improved significantly__

### 3.3 Machine Learning Model : Function 2

In [10]:
def final_func_2(test_data2,y_test):
    
    #Data Preprocessing
    # processing all missing data and combining them in one format
    test_data2.replace(to_replace = ['nan','na','NA','n.a.','?','--','n/a'], value = np.NaN, inplace = True)
    
    #imputing missing values using median imputation
    test_data2 = impute_model(test_data2)
    
    #matching the sampling data
    testdf = get_resampling(test_data2)
    
    #Converting result(meta model) into dataframe
    test_meta = pd.DataFrame(testdf)
    
    #Applying the pretrained model

    filename = 'Model_LR80.sav'

    loaded_model = pickle.load(open(filename,'rb'))

    y_pred_test = loaded_model.predict(test_meta)
    
    cost = get_confusion_matrix(y_test,y_pred_test)
    
    return cost

In [11]:
total_cost = final_func_2(test_data,y_test)

Accuracy : 1.000
Precision : 0.000
Recall : 0.000
f1_score : 0.000
------------------------------------------------------------------------------------------------------------------------
Dimension of Confusion Matrix :  1
------------------------------------------------------------------------------------------------------------------------
Confusion Matrix : 
 [[1]]
------------------------------------------------------------------------------------------------------------------------
Test data contains only 1 type of data
Total cost :  0


__Observations : 1. Function 2 provides us the cost matrix as well precision and recall of prediction of data__

__2. precsion and recall keep on changing depending on number of inputs__

__3. if we consider the entire test data, our total cost is around 21000 using Logistic Regression__

__4. The function can work for dataset from 1 to max no. of rows and cost will keep on increasing__